# MAPPO Traffic Signal Control - Colab Training

本 notebook 用于在 Google Colab 上运行 MAPPO 交通信号控制训练。

**使用前请确保：**
1. 已将最新代码 push 到 Gitee 仓库
2. 运行时类型已设置为 GPU（T4）
3. 已挂载 Google Drive（用于保存训练结果）

## 1. 环境准备

In [ ]:
# 挂载 Google Drive（用于持久化保存训练结果）
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# 安装 SUMO
!apt-get update -qq
!apt-get install -y -qq sumo sumo-tools sumo-doc

import os
os.environ['SUMO_HOME'] = '/usr/share/sumo'
print(f"SUMO_HOME = {os.environ['SUMO_HOME']}")
!sumo --version

In [ ]:
# Clone 项目代码
# 如果是私有仓库，需要在 URL 中加入 token：
# !git clone https://<your_token>@gitee.com/fzzf7478/mappo_traffic_signal.git

%cd /content
!rm -rf mappo_traffic_signal  # 清理旧代码
!git clone https://gitee.com/fzzf7478/mappo_traffic_signal.git
%cd mappo_traffic_signal
!git log --oneline -5

In [ ]:
# 安装 Python 依赖
# 只安装训练必需的包，跳过不必要的（atari, gym_sokoban 等）
!pip install -q easydict tensorboardX torch torchvision torch_geometric \
    scikit-learn pyyaml tabulate sumolib traci cloudpickle lz4 \
    matplotlib seaborn

In [ ]:
# 验证环境
import torch
print(f"PyTorch: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_mem / 1e9:.1f} GB")

import traci
print(f"traci version: {traci.__version__}")

import sumolib
print(f"sumolib OK")

print(f"\nSUMO_HOME: {os.environ.get('SUMO_HOME', 'NOT SET')}")
print("环境验证通过！")

## 2. 训练配置

选择要运行的实验配置。可选：
- `sumo_3roads_mappo_baseline.py` — 3路口 baseline（无注意力）
- `sumo_3roads_mappo_sota.py` — 3路口 sota（含 cross-attention + GCN）
- `sumo_7roads_mappo_baseline.py` — 7路口 baseline
- `sumo_7roads_mappo_sota.py` — 7路口 sota

In [ ]:
# ========== 训练参数配置 ==========

# 选择实验配置（修改这里切换实验）
DING_CONFIG = 'sumo_3roads_mappo_baseline.py'  # 或 sumo_3roads_mappo_sota.py
ENV_CONFIG  = 'sumo_3roads_multi_agent_config.yaml'
SEED = 42
EXP_NAME = 'colab_3roads_baseline_s42'

# 是否将结果保存到 Google Drive
SAVE_TO_DRIVE = True
DRIVE_SAVE_DIR = '/content/drive/MyDrive/MAPPO_results'

# ===================================

PROJECT_ROOT = '/content/mappo_traffic_signal'
DING_CFG_PATH = f'{PROJECT_ROOT}/signal_control/entry/sumo_config/{DING_CONFIG}'
ENV_CFG_PATH  = f'{PROJECT_ROOT}/signal_control/smartcross/envs/{ENV_CONFIG}'

print(f"实验配置: {DING_CONFIG}")
print(f"环境配置: {ENV_CONFIG}")
print(f"随机种子: {SEED}")
print(f"实验名称: {EXP_NAME}")

## 3. 启动训练

In [ ]:
# 启动训练
%cd {PROJECT_ROOT}

!python signal_control/entry/sumo_train \
    -d {DING_CFG_PATH} \
    -e {ENV_CFG_PATH} \
    -s {SEED} \
    --exp-name {EXP_NAME}

## 4. 查看训练结果

In [ ]:
import glob

# 查找实验目录（可能带时间戳后缀）
exp_dirs = sorted(glob.glob(f'{PROJECT_ROOT}/{EXP_NAME}*'))
if exp_dirs:
    exp_dir = exp_dirs[-1]  # 取最新的
    print(f"实验目录: {exp_dir}")
else:
    print("未找到实验目录！")
    exp_dir = None

In [ ]:
# 查看 evaluator 日志
if exp_dir:
    eval_log = f'{exp_dir}/log/evaluator/evaluator_logger.txt'
    if os.path.exists(eval_log):
        with open(eval_log, 'r') as f:
            print(f.read())
    else:
        print("evaluator 日志尚未生成")

In [ ]:
# 查看 learner 日志（最后 50 行）
if exp_dir:
    learner_log = f'{exp_dir}/log/learner/learner_logger.txt'
    if os.path.exists(learner_log):
        with open(learner_log, 'r') as f:
            lines = f.readlines()
            print(f"总行数: {len(lines)}")
            print(''.join(lines[-50:]))
    else:
        print("learner 日志尚未生成")

In [ ]:
# 启动 TensorBoard（在 Colab 中内嵌显示）
if exp_dir:
    %load_ext tensorboard
    %tensorboard --logdir {exp_dir}/tensorboard

## 5. 保存结果到 Google Drive

In [ ]:
import shutil

if SAVE_TO_DRIVE and exp_dir:
    os.makedirs(DRIVE_SAVE_DIR, exist_ok=True)
    dest = os.path.join(DRIVE_SAVE_DIR, os.path.basename(exp_dir))
    if os.path.exists(dest):
        shutil.rmtree(dest)
    shutil.copytree(exp_dir, dest)
    print(f"结果已保存到: {dest}")
else:
    print("跳过保存（SAVE_TO_DRIVE=False 或无实验目录）")

## 6. 批量实验（可选）

如果需要同时跑 baseline 和 sota 对比实验，可以依次运行以下 cell。

In [ ]:
# 批量实验配置
experiments = [
    {
        'ding_config': 'sumo_3roads_mappo_baseline.py',
        'env_config': 'sumo_3roads_multi_agent_config.yaml',
        'exp_name': 'colab_3roads_baseline_s42',
        'seed': 42,
    },
    {
        'ding_config': 'sumo_3roads_mappo_sota.py',
        'env_config': 'sumo_3roads_multi_agent_config.yaml',
        'exp_name': 'colab_3roads_sota_s42',
        'seed': 42,
    },
]

for i, exp in enumerate(experiments):
    print(f"\n{'='*60}")
    print(f"实验 {i+1}/{len(experiments)}: {exp['exp_name']}")
    print(f"{'='*60}\n")
    
    ding_cfg = f"{PROJECT_ROOT}/signal_control/entry/sumo_config/{exp['ding_config']}"
    env_cfg  = f"{PROJECT_ROOT}/signal_control/smartcross/envs/{exp['env_config']}"
    
    !cd {PROJECT_ROOT} && python signal_control/entry/sumo_train \
        -d {ding_cfg} \
        -e {env_cfg} \
        -s {exp['seed']} \
        --exp-name {exp['exp_name']}
    
    # 保存到 Drive
    if SAVE_TO_DRIVE:
        exp_dirs = sorted(glob.glob(f"{PROJECT_ROOT}/{exp['exp_name']}*"))
        if exp_dirs:
            src = exp_dirs[-1]
            dest = os.path.join(DRIVE_SAVE_DIR, os.path.basename(src))
            if os.path.exists(dest):
                shutil.rmtree(dest)
            shutil.copytree(src, dest)
            print(f"已保存: {dest}")

print("\n所有实验完成！")